<a href="https://colab.research.google.com/github/Narendra-Sompalle/GenAIProjects/blob/main/RAG_GraphDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
! pip install --upgrade --quiet langchain langchain-community langchain_groq neo4j langchain_experimental langchain-google-genai

In [70]:
NEO4J_URI=""
NEO4J_USERNAME=""
NEO4J_PASSWORD=""

In [28]:
import os

In [29]:
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [30]:
from langchain_community.graphs import Neo4jGraph

In [31]:
Neo=Neo4jGraph(
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD"),
)


In [32]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [42]:
from langchain_google_genai.llms import ChatGoogleGenerativeAI

In [71]:
llm=ChatGoogleGenerativeAI(temperature=1,model="gemini-2.0-flash",api_key="")

In [46]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [47]:
graph_trans=LLMGraphTransformer(llm=llm)

In [48]:
from langchain_core.documents import Document
text="""Use server-side calls with API keys The most secure way to use your API key is to call the Gemini API from a server-side application where the key can be kept confidential.

Use ephemeral tokens for client-side access (Live API only): For direct client-side access to the Live API, you can use ephemeral tokens. They come with lower security risks and can be suitable for production use. Review ephemeral tokens guide for more information."""

In [49]:
document=Document(page_content=text)

In [51]:
trans=graph_trans.convert_to_graph_documents([document])

In [52]:
trans

[GraphDocument(nodes=[Node(id='Gemini Api', type='Api', properties={}), Node(id='Api Key', type='Key', properties={}), Node(id='Ephemeral Tokens', type='Token', properties={}), Node(id='Live Api', type='Api', properties={})], relationships=[Relationship(source=Node(id='Gemini Api', type='Api', properties={}), target=Node(id='Api Key', type='Key', properties={}), type='USES', properties={}), Relationship(source=Node(id='Live Api', type='Api', properties={}), target=Node(id='Ephemeral Tokens', type='Token', properties={}), type='USES', properties={})], source=Document(metadata={}, page_content='Use server-side calls with API keys The most secure way to use your API key is to call the Gemini API from a server-side application where the key can be kept confidential.\n\nUse ephemeral tokens for client-side access (Live API only): For direct client-side access to the Live API, you can use ephemeral tokens. They come with lower security risks and can be suitable for production use. Review eph

In [53]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [54]:
Neo.query(movie_query)

[]

In [66]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=Neo,verbose=False,allow_dangerous_requests=True)

In [67]:
chain

GraphCypherQAChain(verbose=False, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7cc442f16990>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=1.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.Generati

In [69]:
chain.invoke("give me only director name don't give extra content, who is director of Jumanji")

{'query': "give me only director name don't give extra content, who is director of Jumanji",
 'result': 'Joe Johnston'}